In [1]:
from utils.dataset import MagnaTagATune
import os

In [23]:
DATA_PATH = os.path.join("..", "data", "MagnaTagATune", "samples")
TRAIN_LABELS_PATH = "../data/MagnaTagATune/annotations/train_labels.pkl"
VAL_LABELS_PATH = "../data/MagnaTagATune/annotations/val_labels.pkl"

data = MagnaTagATune(TRAIN_LABELS_PATH, DATA_PATH)

Loading data from ../data/MagnaTagATune/annotations/train_labels.pkl...


In [24]:
print(f'Number of files: {len(data)}')
print("File name of file 0:")
print(data[0][0])
print(f'Samples of shape {data[0][1].shape}:')
print(data[0][1])
print(f'Labels of shape {data[0][2].shape}')
print(data[0][2])

Number of files: 16963
File name of file 0:
train/0/american_bach_soloists-j_s__bach__cantatas_volume_v-01-gleichwie_der_regen_und_schnee_vom_himmel_fallt_bwv_18_i_sinfonia-117-146.npy
Samples of shape torch.Size([10, 1, 34950]):
tensor([[[    0.,     0.,     0.,  ...,  -221.,  -498.,  -191.]],

        [[  209.,   366.,   494.,  ...,  -235.,  -283.,  -138.]],

        [[  -96.,  -388.,  -536.,  ...,   698.,   515.,   624.]],

        ...,

        [[-1510., -1214.,  -812.,  ...,  -416.,  -259.,   -37.]],

        [[  239.,   384.,   467.,  ..., -1524., -1423., -1371.]],

        [[-1308.,  -960.,  -854.,  ...,     0.,     0.,     0.]]])
Labels of shape torch.Size([50])
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
